In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)

In [16]:
def generate(filename):
    df = pd.read_csv(filename,delimiter='\s+',header=None)
    X = df[df.columns[:-1]].values
    y = df[df.columns[-1]].values
    
    num_classes = len(np.unique(y))
    N = len(X)
    
    Y = np.zeros((N,num_classes))
    
    for i in range(N):
        Y[i][y[i]-1] = 1
    
    return X,Y

In [17]:
X_train,Y_train = generate('trainNN.txt')
X_test, Y_test  =  generate('testNN.txt')

In [18]:
class L_Layer_NN():
    def __init__(self,k,lr=0.25,max_epoch=1000):
        self.k = k
        self.L = len(self.k)-1
        self.weights = []
        self.lr = lr
        self.max_epoch = max_epoch
        
    def init_weights(self):
        for i in range(1,len(self.k)):
            self.weights.append(np.random.uniform(0,1,(self.k[i],self.k[i-1]+1)))
    def sigmoid(self,z):
        return 1/(1 + np.exp(-z))

    def derivative(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
            
    def feed_forward(self,x_i):
        self.y = []
        self.v = []
        
        self.y.append(x_i)
        
        for r in range(self.L):
            v_r = np.dot(self.weights[r],self.y[r])
            self.v.append(v_r)
            y_r = self.sigmoid(v_r)
            y_r = np.insert(y_r,0,values=1,axis=0)
            self.y.append(y_r)
    
    def backpropagation(self,y_true):
        self.delta = [0]*self.L
        self.y_hat = self.y[-1]
        self.delta[self.L-1] = np.multiply(self.y_hat[1:]-y_true,self.derivative(self.v[self.L-1]))    
        
        for r in reversed(range(0,self.L-1)):
            e_r = np.dot(self.delta[r+1],self.weights[r+1][:,1:])
            self.delta[r] = np.multiply(e_r,self.derivative(self.v[r]))
    
    def update_weights(self):
        self.y = self.y[:-1]
        for r in range(self.L):
            self.weights[r] = self.weights[r] - self.lr*((self.delta[r].reshape(-1,1))*self.y[r])
    
    def calculate_cost(self,y_hat,y_true):
        return 0.5*np.sum((y_hat-y_true)**2,axis=0)
    
    def fit(self,X,Y):
        self.init_weights() ## initialize weights
        X = np.insert(X,0,values=1,axis=1) ## appending 1 to all x vectors
        
        for epoch in range(self.max_epoch):
            total_cost = 0
            for (x_i,y_i) in zip(X,Y):
                self.feed_forward(x_i)
                self.backpropagation(y_i)
                self.update_weights()
                total_cost += self.calculate_cost(self.y_hat[1:],y_i)
            print("Epoch: ",epoch,total_cost)
            if total_cost < 5:
                break
        return
    
    def predict(self,X,Y):
        correct = 0
        sample_number = 0
        
        X = np.insert(X,0,values=1,axis=1)
        
        for (x_i,y_i) in zip(X,Y):
            self.feed_forward(x_i)
            sample_number += 1
            
            predicted_class = np.argmax(self.y[-1][1:])
            actual_class = np.argmax(y_i)
            
            if predicted_class == actual_class:
                correct += 1
            else:
                print(sample_number,x_i,y_i,actual_class+1,predicted_class+1)
        
        return correct/len(X)

In [19]:
min_max_scaler = MinMaxScaler()  # min max scaler
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

input_neurons = X_train.shape[1]
output_neurons = Y_train.shape[1] 

model = L_Layer_NN([input_neurons,4,5,6,7,output_neurons],lr=1)
model.fit(X_train,Y_train)

Epoch:  0 218.1842166129956
Epoch:  1 197.68425992821943
Epoch:  2 193.0802341925982
Epoch:  3 192.04400569529975
Epoch:  4 191.86808556121505
Epoch:  5 191.79738303460041
Epoch:  6 191.75934695490415
Epoch:  7 191.73542115843094
Epoch:  8 191.71871515545317
Epoch:  9 191.70609820725448
Epoch:  10 191.6959409531805
Epoch:  11 191.68730176365779
Epoch:  12 191.67958405678363
Epoch:  13 191.6723735574
Epoch:  14 191.66535217263507
Epoch:  15 191.65824652571388
Epoch:  16 191.65079173349892
Epoch:  17 191.6426996629408
Epoch:  18 191.63362357392546
Epoch:  19 191.62311004554354
Epoch:  20 191.6105239532604
Epoch:  21 191.59491959624748
Epoch:  22 191.57480093209008
Epoch:  23 191.5476375284021
Epoch:  24 191.5087884034412
Epoch:  25 191.44879265167216
Epoch:  26 191.34526372169339
Epoch:  27 191.1313560533982
Epoch:  28 190.499665264371
Epoch:  29 184.7402763393755
Epoch:  30 132.76647688738848
Epoch:  31 118.32376085591845
Epoch:  32 99.63933735531208
Epoch:  33 75.53516177875709
Epoch: 

In [20]:
model.predict(X_train,Y_train)

1.0

In [21]:
model.predict(X_test,Y_test)

1.0